<a href="https://colab.research.google.com/github/kieutrinhtran/BigData-Homework/blob/main/31221025966_TranThiKieuTrinh_Hive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hướng dẫn thực hành Hive**

In [ ]:
from datetime import datetime
import pytz
print('Tested',datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')))

Tested 2025-04-11 20:53:10.256308+07:00


#Hive with Hadoop
This notebook has all the codes / commands required to install Hadoop and Hive <br>



##Acknowledgements
Hadoop Installation from [Anjaly Sam's Github Repository](https://github.com/anjalysam/Hadoop) <br>
Hive Installation from [PhoenixNAP](https://phoenixnap.com/kb/install-hive-on-ubuntu) website

#1 Hadoop
Hadoop is a pre-requisite for Hive <br>


## 1.1 Download, Install Hadoop

In [ ]:
# The default JVM available at /usr/lib/jvm/java-11-openjdk-amd64/  works for Hadoop
# But gives errors with Hive https://stackoverflow.com/questions/54037773/hive-exception-class-jdk-internal-loader-classloadersappclassloader-cannot
# Hence this JVM needs to be installed
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download the latest version of Hadoop
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz
# Unzip it
# the tar command with the -x flag to extract, -z to uncompress, -v for verbose output, and -f to specify that we’re extracting from a file
!tar -xzf hadoop-3.3.5.tar.gz
#copy  hadoop file to user/local
!mv  hadoop-3.3.5/ /usr/local/

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


## 1.2 Set Environment Variables


In [ ]:
#To set java path, go to /usr/local/hadoop-3.3.0/etc/hadoop/hadoop-env.sh then
#. . . export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/ . . .
#we have used a simpler alternative route using os.environ - it works

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"   # default is changed
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.5/"

# Add Hadoop BIN to PATH
# current_path taken from last command
current_path = os.getenv('PATH')
new_path = current_path+':/usr/local/hadoop-3.3.5/bin/'
os.environ["PATH"] = new_path

## 1.3 Test Hadoop Installation

In [ ]:
# Testing Hadoop with PI generating sample program, should calculate value of pi = 3.14157500000000000000
# pi example
#Uncomment the following line if  you want to test Hadoop with pi example
#!hadoop jar /usr/local/hadoop-3.5.0/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.5.0.jar pi 16 100000
!hdfs dfs -ls

Found 3 items
drwxr-xr-x   - root root       4096 2025-04-09 13:38 .config
-rw-r--r--   1 root root  706533213 2023-03-15 19:35 hadoop-3.3.5.tar.gz
drwxr-xr-x   - root root       4096 2025-04-09 13:38 sample_data


#2 Hive

## 2.1 Download, Install HIVE

In [ ]:
# Download and Unzip the correct version and unzip
!wget -q https://archive.apache.org/dist/hive/hive-3.1.3/apache-hive-3.1.3-bin.tar.gz

!tar xzf apache-hive-3.1.3-bin.tar.gz

## 2.2 Set Environment *Variables*

In [ ]:
# Make sure that the version number is correct and is as downloaded
os.environ["HIVE_HOME"] = "/content/apache-hive-3.1.3-bin"
!echo $HIVE_HOME

# current_path is set from output of previous command
!echo $PATH

current_path = os.getenv('PATH')
new_path = current_path+':/content/apache-hive-3.1.3-bin/bin'
os.environ["PATH"] = new_path
!echo $PATH

!echo $JAVA_HOME
!echo $HADOOP_HOME
!echo $HIVE_HOME

/content/apache-hive-3.1.3-bin
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.5/bin/
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.5/bin/:/content/apache-hive-3.1.3-bin/bin
/usr/lib/jvm/java-8-openjdk-amd64
/usr/local/hadoop-3.3.5/
/content/apache-hive-3.1.3-bin


## 2.3 Set up HDFS Directories

In [ ]:
!hdfs dfs -mkdir /tmp
!hdfs dfs -chmod g+w /tmp
#!hdfs dfs -ls /
!hdfs dfs -mkdir -p /content/warehouse
!hdfs dfs -chmod g+w /content/warehouse
#!hdfs dfs -ls /content/

mkdir: `/tmp': File exists


## 2.4 Initialise HIVE - note and fix errors

In [ ]:
# TYPE this command, do not copy and paste. Non printing characters cause havoc
# There will be two errors, that we will fix
# UNCOMMENT the following line if you WISH TO SEE the errors
#!schematool -initSchema -dbType derby


### 2.4.1 Fix One Warning, One Error
SLF4J is duplicate, need to locate them and remove one <br>
Guava jar version is low

In [ ]:
# locate multiple instances of slf4j ...
!ls $HADOOP_HOME/share/hadoop/common/lib/*slf4j*
!ls $HIVE_HOME/lib/*slf4j*

# removed the logging jar from Hive, retaining the Hadoop jar
!mv /content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar ./

# guava jar needs to above v 20
# https://stackoverflow.com/questions/45247193/nosuchmethoderror-com-google-common-base-preconditions-checkargumentzljava-lan
!ls $HIVE_HOME/lib/gu*

# the one available with Hadoop is better, v 27
!ls $HADOOP_HOME/share/hadoop/hdfs/lib/gu*

# Remove the Hive Guava and replace with Hadoop Guava
!mv $HIVE_HOME/lib/guava-19.0.jar ./

!cp $HADOOP_HOME/share/hadoop/hdfs/lib/guava-27.0-jre.jar $HIVE_HOME/lib/

/usr/local/hadoop-3.3.5//share/hadoop/common/lib/jul-to-slf4j-1.7.36.jar
/usr/local/hadoop-3.3.5//share/hadoop/common/lib/slf4j-api-1.7.36.jar
/usr/local/hadoop-3.3.5//share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar
/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar
/content/apache-hive-3.1.3-bin/lib/guava-19.0.jar
/usr/local/hadoop-3.3.5//share/hadoop/hdfs/lib/guava-27.0-jre.jar


##2.5 Initialize HIVE

In [ ]:
#Type this command, dont copy-paste
# Non printing characters inside the command will give totally illogical errors
!schematool -initSchema -dbType derby

Metastore connection URL:	 jdbc:derby:;databaseName=metastore_db;create=true
Metastore Connection Driver :	 org.apache.derby.jdbc.EmbeddedDriver
Metastore connection User:	 APP
Starting metastore schema initialization to 3.1.0
Initialization script hive-schema-3.1.0.derby.sql

 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 

 

 
 
 

 
 
 


 
 
 

 
 
 


 
 
 
 
 
 
 


 
 
 

 
 
 

 
 

 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 


 



 


 
 
 
 
 
 
 
 
 

 
 
 
 
 
 

 


Initialization script completed
schemaTool completed


## 2.6 Test HIVE
1. Create database
2. Create table
3. Insert data
4. Retrieve data

using command line options as [given here](https://cwiki.apache.org/confluence/display/hive/languagemanual+cli#).

In [ ]:
!hive -e 'create database if not exists praxisDB; show databases;'

Hive Session ID = 42222f64-8130-438d-9681-d6f83cdba348

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 126722fc-ee62-4612-8141-fbdb441a7dca
OK
Time taken: 1.525 seconds
OK
default
praxisdb
Time taken: 0.259 seconds, Fetched: 2 row(s)


In [ ]:
!hive -database praxisdb -e "\
create table if not exists emp2 (name string, age int);\
show tables;\
"

Hive Session ID = dfb24a2e-2bdd-4ceb-a59b-f94c3f00d5ff

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = aab3a4c7-735b-4e51-8237-0c0b3780a1af
OK
Time taken: 1.287 seconds
OK
Time taken: 1.636 seconds
OK
emp2
Time taken: 0.388 seconds, Fetched: 1 row(s)


In [ ]:
!hive -database praxisdb -e "show tables"

Hive Session ID = 7b0bd942-00f8-408c-b9f7-aabdca1cd6c3

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 6cb6d442-6474-4aea-8847-14af9f754581
OK
Time taken: 1.4 seconds
OK
emp2
Time taken: 0.28 seconds, Fetched: 1 row(s)


In [ ]:
!hive -database praxisdb -e "\
insert into emp2 values ('naren', 70);\
insert into emp2 values ('aditya', 49);\
"

Hive Session ID = 873d5860-05ca-4a0b-94e2-4daef660b2f6

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 947f35ba-1e75-48c6-9a8a-350ef5f45ce7
OK
Time taken: 1.983 seconds
Query ID = root_20250411135921_d496fc6d-2dc1-485b-bbf3-a86104fa0cd7
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2025-04-11 13:59:27,965 Stage-1 map = 100%,  reduce = 0%
2025-04-11 13:59:28,971 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local142187874_0001
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by conditio

In [ ]:
!hive -database praxisdb -e "\
select * from emp2;\
select * from emp2 where name = 'aditya';\
"

Hive Session ID = e6085bb7-ec5b-4c49-a033-316fe33e48e5

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = ec32052a-e04f-4b98-8213-3c71f59d91d6
OK
Time taken: 1.295 seconds
OK
naren	70
aditya	49
Time taken: 2.578 seconds, Fetched: 2 row(s)
OK
aditya	49
Time taken: 0.666 seconds, Fetched: 1 row(s)


In [ ]:
# Silent Mode
#!hive -S -database praxisdb -e "select * from emp2 where name = 'aditya'"

## Làm bài tập

In [ ]:
#Tạo CSDL
!hive -e 'create database if not exists ueh_bigdata_31221025966; show databases;'

Hive Session ID = 3750cb33-97bd-481c-ad3b-c276ae386066

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 5128f02e-be7a-4b4e-afdc-dc2fdc501c87
OK
Time taken: 2.487 seconds
OK
default
praxisdb
ueh_bigdata_31221025966
Time taken: 0.436 seconds, Fetched: 3 row(s)


In [ ]:
# Tạo nội dung câu lệnh tạo bảng Hive đúng
create_table_hql = """
CREATE TABLE IF NOT EXISTS Sale_31221025966 (
  InvoiceID STRING,
  Branch STRING,
  City STRING,
  CustomerType STRING,
  Gender STRING,
  ProductLine STRING,
  UnitPrice FLOAT,
  Quantity INT,
  Tax5 FLOAT,
  Total FLOAT,
  `Date` STRING,
  `Time` STRING,
  Payment STRING,
  COGS FLOAT,
  GrossMarginPercentage FLOAT,
  GrossIncome FLOAT,
  Rating FLOAT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
TBLPROPERTIES ("skip.header.line.count"="1");
"""

# Lưu vào file .hql
with open("create_table_Sale_31221025966.hql", "w") as f:
    f.write(create_table_hql)


In [ ]:
!hive -database ueh_bigdata_31221025966 -f create_table_Sale_31221025966.hql

Hive Session ID = a30610ad-bddd-44a9-83ef-9798db248521

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = fb6e085f-f11e-4e97-95b6-4bf53b117a88
OK
Time taken: 1.244 seconds
OK
Time taken: 1.102 seconds


In [ ]:
!hive -database ueh_bigdata_31221025966 -e "SHOW TABLES;"

Hive Session ID = da08103e-afc0-4ad2-988d-cdda7878288d

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 7d1570a1-b235-431a-87b2-74b80016d4c9
FAILED: SemanticException [Error 10072]: Database does not exist: ueh_bigdata_31221025966


In [ ]:
!pip install -q gdown

# File từ Google Drive: https://drive.google.com/file/d/1vP9yiKaNnBxINT-ccGctORuChzO4aSkE/view
# Lấy ID là: 1vP9yiKaNnBxINT-ccGctORuChzO4aSkE
!gdown --id 1vP9yiKaNnBxINT-ccGctORuChzO4aSkE --output SuperMarketSales.csv


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1vP9yiKaNnBxINT-ccGctORuChzO4aSkE
To: /content/SuperMarketSales.csv
100% 131k/131k [00:00<00:00, 68.3MB/s]


In [ ]:
#Tải file CSV lên HDFS
# Tạo thư mục nếu cần
!hdfs dfs -mkdir -p /tmp/sales_data

# Tải file lên HDFS
!hdfs dfs -put -f SuperMarketSales.csv /tmp/sales_data/

#Load dữ liệu vào bảng Hive
!hive -database ueh_bigdata_31221025966 -e "LOAD DATA INPATH '/tmp/sales_data/SuperMarketSales.csv' INTO TABLE Sale_31221025966;"


Hive Session ID = 4568744d-b7c4-413e-9ae7-4156622d7024

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 14d14f23-a6f7-4390-8419-410113dde37d
OK
Time taken: 1.375 seconds
Loading data to table ueh_bigdata_31221025966.sale_31221025966
OK
Time taken: 1.6 seconds


In [ ]:
#Kiểm tra dữ liệu
!hive -database ueh_bigdata_31221025966 -e "SELECT * FROM Sale_31221025966 LIMIT 10;"


Hive Session ID = 2cdb340f-4d20-408c-a90e-b4d70429bf8f

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = b0c51805-8638-4abc-8dce-25b813c9def1
OK
Time taken: 1.287 seconds
OK
750-67-8428	A	Yangon	Member	Female	Health and beauty	74.69	7	26.1415	548.9715	1/5/2019	13:08	Ewallet	522.83	4.7619047	26.1415	9.1
226-31-3081	C	Naypyitaw	Normal	Female	Electronic accessories	15.28	5	3.82	80.22	3/8/2019	10:29	Cash	76.4	4.7619047	3.82	9.6
631-41-3108	A	Yangon	Normal	Male	Home and lifestyle	46.33	7	16.2155	340.5255	3/3/2019	13:23	Credit card	324.31	4.7619047	16.2155	7.4
123-19-1176	A	Yangon	Member	Male	Health and beauty	58.22	8	23.288	489.048	1/27/2019	20:33	Ewallet	465.76	4.7619047	23.288	8.4
373-73-7910	A	Yangon	Normal	Male	Sports and travel	86.31	7	30.2085	634.3785	2/8/2019	10:37	Ewallet	604.17	4.7619047	30.2085	5.3
699-14-3026	C	Naypyitaw	Normal	Male	Electronic accessories	85.39	7	29.8

In [ ]:
#Đếm số dòng đã load vào bảng:
!hive -S -database ueh_bigdata_31221025966 -e "SELECT COUNT(*) FROM Sale_31221025966;"


Hive Session ID = df1a7024-d95b-4512-8b3e-cef87f889cec
Hive Session ID = 43ec823f-54cb-4933-9f36-757bfd273952
FAILED: SemanticException [Error 10072]: Database does not exist: ueh_bigdata_31221025966


### Viết câu truy vấn

#### Group theo City, có bao nhiêu đơn hàng, Tổng tiền, đơn hàng lớn nhất, đơn hàng nhỏ nhất và trung bình điểm Rating

In [ ]:
#Group theo City, có bao nhiêu đơn hàng, Tổng tiền, đơn hàng lớn nhất, đơn hàng nhỏ nhất và trung bình điểm Rating

!hive -database ueh_bigdata_31221025966 -e "\
SELECT City, \
       COUNT(*) AS SoLuongDonHang, \
       SUM(Total) AS TongTien, \
       MAX(Total) AS DonHangLonNhat, \
       MIN(Total) AS DonHangNhoNhat, \
       ROUND(AVG(Rating), 2) AS DiemTrungBinh \
FROM Sale_31221025966 \
GROUP BY City;"


Hive Session ID = f8d689c6-5474-462a-b6b8-1cf41e00f65d

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 22937474-951a-4384-8901-02847921f8f5
OK
Time taken: 1.247 seconds
Query ID = root_20250411143107_944fc8df-4fe2-42ef-924e-aa600d701946
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2025-04-11 14:31:14,814 Stage-1 map = 100%,  reduce = 0%
2025-04-11 14:31:15,826 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local1028287766_0001
MapReduce Jobs Launched: 
Stage-Stage-1:  HDFS Read: 0 HDF

#### Liệt kê 10 đơn hàng có giá trị cao nhất

In [ ]:
#Liệt kê 10 đơn hàng có giá trị cao nhất

!hive -database ueh_bigdata_31221025966 -e "\
SELECT InvoiceID, City, Branch, CustomerType, Gender, ProductLine, Total, Rating \
FROM Sale_31221025966 \
ORDER BY Total DESC \
LIMIT 10;"


Hive Session ID = 99ed7bde-67b0-474c-bc77-1e1fa426ea50

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 96b99d5f-a5d5-477c-8a77-6beef09782e1
OK
Time taken: 1.278 seconds
Query ID = root_20250411143922_86d8171e-3db4-4734-924f-b7553394c79e
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2025-04-11 14:39:29,263 Stage-1 map = 0%,  reduce = 0%
2025-04-11 14:39:30,277 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local815717641_0001
MapReduce Jobs Launched: 
Stage-Stage-1:  HDFS Read: 0 HDFS Write: 0 SUCCESS
Tot

#### Tính toán tổng doanh thu và lợi nhuận từ các đơn hàng đã hoàn thành.

In [ ]:
#Tính toán tổng doanh thu và lợi nhuận từ các đơn hàng đã hoàn thành.

!hive -database ueh_bigdata_31221025966 -e "\
SELECT ROUND(SUM(Total), 2) AS TongDoanhThu, \
       ROUND(SUM(GrossIncome), 2) AS TongLoiNhuan \
FROM Sale_31221025966;"


Hive Session ID = be3ec5ac-e991-449a-8a84-e17ab1660552

Logging initialized using configuration in jar:file:/content/apache-hive-3.1.3-bin/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 454ac7e6-02fe-444e-b859-c08f52f5f07d
OK
Time taken: 1.396 seconds
Query ID = root_20250411143947_294cbd2d-e192-4ec6-96b3-a56c3540e162
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2025-04-11 14:39:54,844 Stage-1 map = 100%,  reduce = 0%
2025-04-11 14:39:55,856 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local1160734477_0001
MapReduce Jobs Launched: 
Stage-Stage-1:  HDFS Read: 0 HDFS Write: 0 SUCCESS


# 3.0 Bulk Data - Super Store Data

In [ ]:
#Get Data File
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'
!head SS_Orders.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United Stat

In [ ]:
!hive -S -database praxisDB -e "\
drop table if exists ss_order;\
CREATE TABLE IF NOT EXISTS ss_order (\
    RowID smallint,\
    OrderID char(14),OrderDate string,\
    ShipDate string,ShipMode varchar(16),\
    CustomerID char(8),CustomerName varchar(30),Segment varchar(20),\
    Country varchar(30),City varchar(30),State varchar(30),PostalCode char(5),Region varchar(15) ,\
    ProductID varchar(20), Category varchar(40), SubCategory varchar(40), ProductName varchar(200), \
    Sales decimal(8,2), Quantity smallint, Discount decimal(4,2), Profit decimal(8,2) \
) row format delimited fields terminated by ','; \
describe ss_order;\
"

Hive Session ID = 398c01c8-7242-43f0-aff8-3bc77a00f2a8
Hive Session ID = 991d65cf-5e20-46be-90b7-6594752d9f78
rowid               	smallint            	                    
orderid             	char(14)            	                    
orderdate           	string              	                    
shipdate            	string              	                    
shipmode            	varchar(16)         	                    
customerid          	char(8)             	                    
customername        	varchar(30)         	                    
segment             	varchar(20)         	                    
country             	varchar(30)         	                    
city                	varchar(30)         	                    
state               	varchar(30)         	                    
postalcode          	char(5)             	                    
region              	varchar(15)         	                    
productid           	varchar(20)         	                    
category

In [ ]:
#remove the CRLF character from the end of the row if it exists
#!sed 's/[^a-zA-Z0-9,\/\.\-]/ /g' SS_Orders.csv > datafile.csv
!sed 's/\r//' /content/SS_Orders.csv > datafile.csv
#!sed 's/\r//' /content/eCommerce_02PC_2021.csv > datafile.csv
# remove the first line containing headers from the file
!sed -i -e "1d" datafile.csv
!head datafile.csv

1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031
5,US-2015-108966,11/10/2015,18/10/2015,Standard Class

In [ ]:
!hive -S -database praxisdb -e "\
TRUNCATE TABLE ss_order;\
LOAD DATA LOCAL INPATH 'datafile.csv' INTO TABLE ss_order;\
"

Hive Session ID = d56b0286-e0c3-4b77-b406-7e9a8290e861
Hive Session ID = 29df1d3a-06b9-4962-a2bf-61bbc8afa133


In [ ]:
!hive -S -database praxisdb -e "\
select count(*) from ss_order;\
select * from ss_order limit 10;\
"

Hive Session ID = 3a34d10e-692a-426d-b515-a30e12266982
Hive Session ID = c36111f0-f279-43d0-a3e8-1876a26ce5ae
9994
1	CA-2016-152156	08/11/2016	11/11/2016	Second Class	CG-12520	Claire Gute	Consumer	United States	Henderson	Kentucky	42420	South	FUR-BO-10001798	Furniture	Bookcases	Bush Somerset Collection Bookcase	261.96	2	0.00	41.91
2	CA-2016-152156	08/11/2016	11/11/2016	Second Class	CG-12520	Claire Gute	Consumer	United States	Henderson	Kentucky	42420	South	FUR-CH-10000454	Furniture	Chairs	"Hon Deluxe Fabric Upholstered Stacking Chairs	NULL	731	3.00	0.00
3	CA-2016-138688	12/06/2016	16/06/2016	Second Class	DV-13045	Darrin Van Huff	Corporate	United States	Los Angeles	California	90036	West	OFF-LA-10000240	Office Supplies	Labels	Self-Adhesive Address Labels for Typewriters by Universal	14.62	2	0.00	6.87
4	US-2015-108966	11/10/2015	18/10/2015	Standard Class	SO-20335	Sean O'Donnell	Consumer	United States	Fort Lauderdale	Florida	33311	South	FUR-TA-10000577	Furniture	Tables	Bretford CR4500 Series

In [ ]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')))

signed off at   2025-04-11 21:43:04.128313+07:00
